# Pandas basics 

Hi! In this programming assignment you need to refresh your `pandas` knowledge. You will need to do several [`groupby`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)s and [`join`]()`s to solve the task. 

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
DATA_FOLDER = './data/'

transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))

The dataset we are going to use is taken from the competition, that serves as the final project for this course. You can find complete data description at the [competition web page](https://www.kaggle.com/c/competitive-data-science-final-project/data). To join the competition use [this link](https://www.kaggle.com/t/1ea93815dca248e99221df42ebde3540).

# Task

Let's start with a simple task. 

<ol start="0">
  <li><b>Print the shape of the loaded dataframes and use [`df.head`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html) function to print several rows. Examine the features you are given.</b></li>
</ol>

In [ ]:
print(f"Transaction shape: {transactions.shape}, head is:")
transactions.head()

In [ ]:
print(f"Items shape: {items.shape}, head is:")
items.head()

In [ ]:
print(f"Item categories shape: {item_categories.shape}, head is:")
item_categories.head()

In [ ]:
print(f"Shops shape: {shops.shape}, head is:")
shops.head()

Now use your `pandas` skills to get answers for the following questions. 
The first question is:

1. ** What was the maximum total revenue among all the shops in September, 2014?** 


* Hereinafter *revenue* refers to total sales minus value of goods returned.

*Hints:*

* Sometimes items are returned, find such examples in the dataset. 
* It is handy to split `date` field into [`day`, `month`, `year`] components and use `df.year == 14` and `df.month == 9` in order to select target subset of dates.
* You may work with `date` feature as with strings, or you may first convert it to `pd.datetime` type with `pd.to_datetime` function, but do not forget to set correct `format` argument.

In [ ]:
transactions['item_value'] = transactions.item_price * transactions.item_cnt_day
transactions.datetime = pd.to_datetime(transactions.date, format="%d.%m.%Y")

mask = (transactions.datetime >= '2014-09-01') & (transactions.datetime <= '2014-09-30')

max_revenue = transactions.loc[mask].groupby("shop_id").item_value.sum().max()
max_revenue

Great! Let's move on and answer another question:

<ol start="2">
  <li><b>What item category generated the highest revenue in summer 2014?</b></li>
</ol>

* Submit `id` of the category found.
    
* Here we call "summer" the period from June to August.

*Hints:*

* Note, that for an object `x` of type `pd.Series`: `x.argmax()` returns **index** of the maximum element. `pd.Series` can have non-trivial index (not `[1, 2, 3, ... ]`).

In [ ]:
transactions.datetime = pd.to_datetime(transactions.date, format="%d.%m.%Y")
mask = (transactions.datetime >= '2014-06-01') & (transactions.datetime <= '2014-08-31')

june_august_transactions = transactions.loc[mask]
joined_df = june_august_transactions[["item_id", "item_value"]] \
    .join(items.set_index("item_id"), on="item_id", how='left')
revenue_per_cat = joined_df.groupby(by="item_category_id")['item_value'].sum().reset_index()

category_id_with_max_revenue = revenue_per_cat.loc[revenue_per_cat['item_value'].argmax()].item_category_id
category_id_with_max_revenue

<ol start="3">
  <li><b>How many items are there, such that their price stays constant (to the best of our knowledge) during the whole period of time?</b></li>
</ol>

* Let's assume, that the items are returned for the same price as they had been sold.

In [ ]:
unique_price = transactions.groupby("item_id")["item_price"].nunique()

num_items_constant_price = unique_price.loc[unique_price == 1].count()
num_items_constant_price

Remember, the data can sometimes be noisy.

<ol start="4">
  <li><b>What was the variance of the number of sold items per day sequence for the shop with `shop_id = 25` in December, 2014? Do not count the items, that were sold but returned back later.</b></li>
</ol>

* Fill `total_num_items_sold` and `days` arrays, and plot the sequence with the code below.
* Then compute variance. Remember, there can be differences in how you normalize variance (biased or unbiased estimate, see [link](https://math.stackexchange.com/questions/496627/the-difference-between-unbiased-biased-estimator-variance)). Compute ***unbiased*** estimate (use the right value for `ddof` argument in `pd.var` or `np.var`). 
* If there were no sales at a given day, ***do not*** impute missing value with zero, just ignore that day

In [ ]:
from datetime import datetime

shop_id = 25

transactions['datetime'] = pd.to_datetime(transactions.date, format="%d.%m.%Y")
date_mask = (transactions.datetime >= '2014-12-01') & (transactions.datetime <= '2014-12-31')
shop_mask = transactions.shop_id == shop_id
december_shop_trans = transactions.loc[date_mask & shop_mask]

#sold_items = december_shop_trans.loc[december_shop_trans["item_cnt_day"] > 0]
#returned_items = december_shop_trans.loc[december_shop_trans["item_cnt_day"] < 0]

#joined_df = sold_items.join(returned_items.set_index(["datetime", "item_id"])["item_cnt_day"], 
#                                               on=["datetime","item_id"], how="left", 
#                            lsuffix="_sold", rsuffix="_returned")

#joined_df["total_cnt"] = joined_df["item_cnt_day_sold"] + joined_df["item_cnt_day_returned"].fillna(0)

#item_sold_per_day = joined_df.reset_index().groupby(by="datetime")["total_cnt"].sum()

total_num_items_sold = december_shop_trans.groupby("datetime")["item_cnt_day"].sum()
days = [d.astype(datetime) for d in total_num_items_sold .index.values]

# Plot it
plt.plot(days, total_num_items_sold)
plt.ylabel('Num items')
plt.xlabel('Day')
plt.title("Daily revenue for shop_id = 25")
plt.show()

total_num_items_sold_var = np.var(total_num_items_sold.values, ddof=1)
total_num_items_sold_var

Well done! :)